# Variational optimization of quantum many-body Hamiltonians with number-conserving circuits
#### Parallelized code

In [2]:
# import necessary libraries and modules
from Ansatzes_Hamiltonians_class import AnsatzCircuit
from Ansatzes_Hamiltonians_class import XXZ, HCBH_ladder_gauged as HCBH
from qiskit.algorithms.optimizers import COBYLA, ADAM
from qiskit.quantum_info import SparsePauliOp
import numpy as np

In [3]:
# Set the parameters for the circuit and the Hamiltonian model

#num_sites = 4
#num_particles = 2
#num_params_per_gate = 2
#gate_class = "A"   # list = ["A", "B", "G"]. # A and B have two parameters, and G has four parameters 
#entangle_initial_state="no"   # no, for initial product state, "yes" for entangled state


# fixed for all simulations 
num_params_per_gate = 2
num_trials = 10  
num_opt_steps = 1000

# model to run
spin_model = "HCBH_nogauge"   # list = ["XXZ", "XY", "HCBH_nogauge", "HCBH_uniform", "HCBH_staggered"]
circ_type = "LC"  # list = ["BWC", "LC"]
initial_ent_value = "yes"  # list = ["no", "yes"]

In [3]:
# the estimator primitive to compute expectation value
from qiskit.primitives import Estimator
exact_estimator = Estimator()

# classical optimizer to update (classical) parameters
#optimizer = COBYLA(maxiter=num_opt_steps, disp=False, tol = 10**-8)
#optimizer = ADAM(maxiter=num_opt_steps, lr=0.01)
#optimizer = ADAM()

optimizer = COBYLA()

In [4]:
# import the ansatz circuit

# define the quantum circuit
def build_circuit(num_particles, num_sites, gate_class="A", entangle_initial_state="no", circ_type='BWC', num_params_per_gate=2):

    global num_params
    
    qc = AnsatzCircuit(
        num_particles,
        num_sites,
        num_params_per_gate,
        gate_class,
        entangle_initial_state, 
        circ_type  
    )
    
    if (qc.dim_subspace < 10):
        print("The quantum circuit: ")
        print(qc.draw())
    
    num_params = qc.num_parameters
    
    # decompose circuit before running
    qc_elem = qc.decompose().decompose() 

    return  qc_elem

In [5]:
# Define the model Hamiltonian
def get_Hamiltonian(num_sites, spin_model):
    global ref_value 

    if spin_model == "XXZ":
        amp = 1
        H_obj = XXZ(num_sites, amp)                
    elif spin_model == "XY":        
        amp = 0
        H_obj = XXZ(num_sites, amp)   # XY = XXZ if ZZ term is turned off
    elif spin_model == "HCBH_nogauge":
        alpha_gauge = 0 
        H_obj = HCBH(num_sites, alpha_gauge)
    elif spin_model == "HCBH_uniform":
        alpha_gauge = 0.5 
        H_obj = HCBH(num_sites, alpha_gauge)
    elif spin_model == "HCBH_staggered":
        alpha_gauge = 0.3 
        H_obj = HCBH(num_sites, alpha_gauge)

    
    H = H_obj.Hamiltonian()
    H_op = SparsePauliOp.from_list(H)

    # get the reference ground state energy for small system sizes
    ref_value = H_obj.get_minimum_eigenvalue(H_op.to_matrix())
    print(f"\n Exact ground state energy: {ref_value}")
    
    return H_op

## Optimization (exact simulation)

In [6]:
# initial and/or input state
import os
from scipy.io import loadmat, savemat, whosmat

def check_for_file(ham_model, num_particles, num_sites, gate_class, entangle_initial_state, num_opt_steps):    
    
    global filename, counts_trials, values_trials, rel_errs_trials, resume_trials, elapsed_time, sim_data

    filename = ham_model + "_" + str(num_particles) + "_" + str(num_sites) + "_" + "sites" + "_" + gate_class + "_" \
                    + entangle_initial_state + "_" + "ent" + "_" + circ_type

    if os.path.exists(filename):    
        print(f"File exists. Will load old data to continue simulation on worker {os.getpid()}\n")

        sim_data = loadmat(filename)        

        # recall the output arrays 
        counts_trials = sim_data['counts_trials'][0]
        values_trials = sim_data['values_trials'][0]
        rel_errs_trials = sim_data['rel_errs_trials'][0]
        resume_trials = sim_data['resume_trials'][0][0]
        elapsed_time = sim_data['elapsed_time'][0][0]

    else:
        # create a new file/sim
        print(f"\nStarting a fresh calculation on worker {os.getpid()}")

        # save the optimized energy values for all iterations and for the # of trials
        counts_trials = np.zeros([num_trials], dtype=object)
        values_trials = np.zeros([num_trials], dtype=object)
        rel_errs_trials = np.zeros([num_trials], dtype=object)

        resume_trials = 0
        elapsed_time = 0

        sim_data = {'num_particles' : num_particles,
                    'num_sites' : num_sites,
                    'gate_class' : gate_class,
                    'entangle_initial_state' : entangle_initial_state,                  
                    'resume_trials' : resume_trials,
                    'num_trials' : num_trials,
                    'num_opt_steps' : num_opt_steps,
                    'elapsed_time' : elapsed_time,
                    'count_trials' : counts_trials,
                    'values_trials' : values_trials,
                    'rel_errs_trials' : rel_errs_trials
                    }

In [7]:
# data preprocessing before plotting

def postprocess(counts_trials, values_trials, rel_errs_trials, num_opt_steps, num_trials):
    """ compute averages of energy expectation values and rel. error per iteration """     

    ## First, repackage data into an a 'proper' 2D array from a 1-D array of 1-D array objects
    counts_trials_mat = np.zeros((num_opt_steps, num_trials))
    values_trials_mat = np.zeros((num_opt_steps, num_trials))
    relerr_trials_mat = np.zeros((num_opt_steps, num_trials))
    
    for i in range(num_trials):
        counts_trials_mat[0:num_opt_steps, i] = counts_trials[i]
        values_trials_mat[0:num_opt_steps, i] = values_trials[i]
        relerr_trials_mat[0:num_opt_steps, i] = rel_errs_trials[i]

    ## compute averages 
    avg_energy_trials = np.zeros(num_opt_steps)
    avg_relerr_trials = np.zeros(num_opt_steps)
    
    for i in range(num_opt_steps):     
        num_trials_eff = np.count_nonzero(counts_trials_mat[i,:])   # number of trials effective
        if num_trials_eff != 0:
            avg_energy_trials[i] = sum(values_trials_mat[i, :])/num_trials_eff
            avg_relerr_trials[i] = sum(relerr_trials_mat[i, :])/num_trials_eff
    
    return avg_energy_trials, avg_relerr_trials

In [25]:
# Main loop cell
from qiskit.algorithms.minimum_eigensolvers import VQE
from timeit import default_timer as timer 

# define the callback function to store intermediate result during optimization
def store_intermediate_result(eval_count, parameters, mean, std):                  
        counts[eval_count-1] = eval_count
        values[eval_count-1] = mean    
        rel_err_val = np.abs((mean-ref_value)/ref_value)
        rel_errs[eval_count-1] = rel_err_val


# the main VQE loop
#def vqe_main(num_particles, num_sites, gate_class, entangle_initial_state, circ_type, ham_model, max_opt_step):        
def vqe_main(argin):
        
      # manual unpacking of input arguments
      num_particles=argin[0]
      num_sites=argin[1]
      gate_class=argin[2]
      entangle_initial_state=argin[3]
      circ_type=argin[4]
      ham_model=argin[5]
      num_opt_steps=argin[6]
      # set the maximum optimization step for this task
      optimizer.__init__(maxiter=num_opt_steps)
      
      # start 
      start_time = timer()  
      
      global elapsed_time      
      
      # build circuit and get Hamiltonian
      qc_elem = build_circuit(num_particles, num_sites, gate_class, entangle_initial_state, circ_type)
      H_op = get_Hamiltonian(num_sites, ham_model)
      check_for_file(ham_model, num_particles, num_sites, gate_class, entangle_initial_state, num_opt_steps)

      # main algorithm loop                         
      for i in range(resume_trials, num_trials): 
          rng = np.random.default_rng(seed=i+1)
          starting_point = rng.uniform(-np.pi, np.pi, num_params)
          
          # define callback
          global counts, values, rel_errs
          counts = np.zeros(num_opt_steps, dtype=int); values = np.zeros(num_opt_steps); rel_errs = np.zeros(num_opt_steps)            
          
          # instantiate and run VQE
          vqe = VQE(exact_estimator, qc_elem, optimizer=optimizer, initial_point = starting_point, callback=store_intermediate_result)
          result = vqe.compute_minimum_eigenvalue(H_op)
          #Energy_estimate = result.eigenvalue.real                        
          
          # save results for each circuit
          counts_trials[i] = counts
          values_trials[i] = values
          rel_errs_trials[i] = rel_errs
          if np.mod(i,5)==0: 
              print(f"Worker {os.getpid()} with {gate_class} gate: Done {i+1} trial samples")                                  
              end_time = timer()
              elapsed_time = elapsed_time + (end_time - start_time)
              start_time = end_time
              # save data at this point
              sim_data.update({
                    'counts_trials'  : counts_trials, 
                    'values_trials'  : values_trials,  
                    'rel_errs_trials' : rel_errs_trials,
                    'resume_trials' : i,                
                    'elapsed_time' : elapsed_time/3600
                    })
              savemat(filename, sim_data)
      
      print(f"Worker {os.getpid()} using {gate_class} gate: Done {i+1} trial samples")          
      
      # Compute average energy and average relative error
      print(f"\nWorker {os.getpid()} with {gate_class}: Post-processing: Computing averages.")
      avg_energy_trials, avg_relerr_trials = postprocess(counts_trials, values_trials, rel_errs_trials, num_opt_steps, num_trials)
      print(f'Worker {os.getpid()} with {gate_class}: last avg. energy :', avg_energy_trials[num_opt_steps-1])
      print(f'Worker {os.getpid()} with {gate_class}: last avg. rel. error :', avg_relerr_trials[num_opt_steps-1])        
      
      # compute the total time taken for the simulation
      end_time = timer()
      elapsed_time = (elapsed_time + (end_time - start_time))
      print(f"\nWorker {os.getpid()} with {gate_class}: Total time taken for simulation = ", elapsed_time, " hours")

      # finally, save all simulation data to disk
      print(f"\r\nWorker {os.getpid()} with {gate_class} gate: === Saving final data to disk === ")
      
      sim_data.update({
            'num_opt_steps' : num_opt_steps,
            'optimizer' : optimizer,
            'counts_trials'  : counts_trials, 
            'values_trials'  : values_trials,  
            'rel_errs_trials' : rel_errs_trials,
            'Exact GS energy' : ref_value,
            'elapsed_time' : elapsed_time/3600,
            'avg_energy_trials' : avg_energy_trials, 
            'avg_relerr_trials' : avg_relerr_trials,
            'resume_trials' : i
            })
      savemat(filename, sim_data)
      
      print(f"\rWorker {os.getpid()} with {gate_class} gate: Done...Elapsed time = ", elapsed_time, " hours\n")

In [26]:
# list of jobs to do for this session

# input ordering 
    #   (num_particles, num_sites, gate_class, entangled_initial_state, circ_type, ham_model, max_opt_step)

# XXZ models
job_list_1 = [
    (2, 4, "A", "yes", "LC", "XXZ", 1000),
    (2, 4, "B", "yes", "LC", "XXZ", 1000),
    (2, 4, "G", "yes", "LC", "XXZ", 1000),
    (2, 4, "A", "yes", "BWC", "XXZ", 1000),
    (2, 4, "B", "yes", "BWC", "XXZ", 1000),
    (2, 4, "G", "yes", "BWC", "XXZ", 1000),
    (3, 6, "A", "yes", "LC", "XXZ", 3500),
    (3, 6, "B", "yes", "LC", "XXZ", 3500),
    (3, 6, "G", "yes", "LC", "XXZ", 3500),
    (3, 6, "A", "yes", "BWC", "XXZ", 3500),
    (3, 6, "B", "yes", "BWC", "XXZ", 3500),
    (3, 6, "G", "yes", "BWC", "XXZ", 3500),
    (4, 8, "A", "yes", "LC", "XXZ", 10000),
    (4, 8, "B", "yes", "LC", "XXZ", 10000),
    (4, 8, "G", "yes", "LC", "XXZ", 10000),
    (4, 8, "A", "yes", "BWC", "XXZ", 10000),
    (4, 8, "B", "yes", "BWC", "XXZ", 10000),
    (4, 8, "G", "yes", "BWC", "XXZ", 10000)
]

# XY models
job_list_2 = [
    (2, 4, "A", "yes", "LC", "XY", 1000),
    (2, 4, "B", "yes", "LC", "XY", 1000),
    (2, 4, "G", "yes", "LC", "XY", 1000),
    (2, 4, "A", "yes", "BWC", "XY", 1000),
    (2, 4, "B", "yes", "BWC", "XY", 1000),
    (2, 4, "G", "yes", "BWC", "XY", 1000),
    (3, 6, "A", "yes", "LC", "XY", 3500),
    (3, 6, "B", "yes", "LC", "XY", 3500),
    (3, 6, "G", "yes", "LC", "XY", 3500),
    (3, 6, "A", "yes", "BWC", "XY", 3500),
    (3, 6, "B", "yes", "BWC", "XY", 3500),
    (3, 6, "G", "yes", "BWC", "XY", 3500),
    (4, 8, "A", "yes", "LC", "XY", 10000),
    (4, 8, "B", "yes", "LC", "XY", 10000),
    (4, 8, "G", "yes", "LC", "XY", 10000),
    (4, 8, "A", "yes", "BWC", "XY", 10000),
    (4, 8, "B", "yes", "BWC", "XY", 10000),
    (4, 8, "G", "yes", "BWC", "XY", 10000)
]


# HCBH models with zero gauge field
job_list_3 = [
    (4, 8, "A", "yes", "LC", "HCBH_nogauge", 10000),
    (4, 8, "B", "yes", "LC", "HCBH_nogauge", 10000),
    (4, 8, "G", "yes", "LC", "HCBH_nogauge", 10000),
    (4, 8, "A", "yes", "BWC", "HCBH_nogauge", 10000),
    (4, 8, "B", "yes", "BWC", "HCBH_nogauge", 10000),
    (4, 8, "G", "yes", "BWC", "HCBH_nogauge", 10000)
]

# HCBH models with staggered gauge field
job_list_4 = [
    (4, 8, "A", "yes", "LC", "HCBH_staggered", 10000),
    (4, 8, "B", "yes", "LC", "HCBH_staggered", 10000),
    (4, 8, "G", "yes", "LC", "HCBH_staggered", 10000),
    (4, 8, "A", "yes", "BWC", "HCBH_staggered", 10000),
    (4, 8, "B", "yes", "BWC", "HCBH_staggered", 10000),
    (4, 8, "G", "yes", "BWC", "HCBH_staggered", 10000)
]


# @@@ Do the simulations of HCBH with 3 on 8 differently. 
# Get the exact GS energy from MATLAB code and modify this code a lil' bit

In [27]:
# parallelizing the run
import multiprocess
from multiprocess import Process
import os

#if __name__ == '__main__':    
    
# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list_1)            
    

The quantum circuit: The quantum circuit: The quantum circuit: 


          ┌────┐      ┌───────────────┐                 ┌───────────────┐»
q_0: ─────┤0   ├──────┤0              ├─────────────────┤0              ├»
     ┌───┐│  B │┌────┐│  B(θ[0],𝜙[0]) │┌───────────────┐│  B(θ[3],𝜙[3]) │»
q_1: ┤ X ├┤1   ├┤0   ├┤1              ├┤0              ├┤1              ├»
     └───┘└────┘│  B │└─────┬────┬────┘│  B(θ[1],𝜙[1]) │├───────────────┤»
q_2: ───────────┤1   ├──────┤0   ├─────┤1              ├┤0              ├»
     ┌───┐      └────┘      │  B │     └───────────────┘│  B(θ[2],𝜙[2]) │»
q_3: ┤ X ├──────────────────┤1   ├──────────────────────┤1              ├»
     └───┘                  └────┘                      └───────────────┘»
«                                       
«q_0: ──────────────────────────────────
«     ┌───────────────┐                 
«q_1: ┤0              ├─────────────────
«     │  B(θ[4],𝜙[4]) │┌───────────────┐
«q_2: ┤1              ├┤0              ├
«     └──────

In [ ]:
# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list_2)            

In [ ]:
# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list_3)            

In [ ]:
# create a pool of workers        
pool = multiprocess.Pool()
res_from_workers = pool.map(vqe_main, job_list_4)            